In [3]:
# 호텔 기본 정보 추가하기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser


def moneyTrimmer(input_string):
    
    '''
    Hotel Price data would look like "$263 - $505".
    This method change the one to 263. (get minimum price)
    '''
    
    temp_list = input_string.split(" - ")
    return (temp_list[0])[1:].replace(",", "")


def hotelInformationGetter(input_data, offset):
    
    '''
    Get hotel data and extract information.
    '''
    
    # 파라미터 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")
    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
    api_url = ("/hotels/list?offset="
                 +str(offset)
                 +"&currency=USD&child_rm_ages=7%252C10&limit=30&checkin=2020-01-08&order=asc&lang=en_US&sort=recommended&nights=1&location_id="
                 + str(location_id)
                 +"&adults=1&rooms=1")
    
    # &hotel_class=5 -> 5등급만 뽑기
    #print(api_url)
    
    conn.request("GET", api_url, headers=headers)
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    hotels_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            hotel_data = []
            hotel_data.append(one_data.get("name"))
            sub_category = one_data.get("subcategory_type")
            sub_categorys = ["hotel", "small_hotel", "hostel", "guest_house", "villa"]
            if not sub_category in sub_categorys:
                continue
            else:
                hotel_data.append(sub_category)                
            hotel_data.append(int(float(one_data.get("hotel_class"))))
            hotel_data.append(int(moneyTrimmer(one_data.get("price"))))
            hotel_data.append(cityId) # cityId
            hotels_data.append(hotel_data)
        except AttributeError as e:
            continue

    return hotels_data

def sqlController(hotel_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password,
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Hotel"
    sql_insert_hotel_information = "INSERT INTO City (continent, country, name) VALUES('Europe', 'France', 'Paris')"
    sql_insert_hotels_information = "INSERT INTO Hotel (name, category, subCategory, cost, CityId) VALUES(%s, %s, %s, %s, %s)"
    sql_delete = "DELETE FROM Hotel"
    sql_update_price = "UPDATE Hotel SET cost = %s WHERE name = %s"
    
    # 호텔 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    hotel_data = curs.fetchall() 
    #print("hotel_data", hotel_data) # 지금까지 DB에 있는 호텔 데이터
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 호텔 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_hotel_data = []
    overlapped_hotel_cost_data = []
    
    for info in hotel_info_list:
        if not dataOverlapChecker(hotel_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_hotel_data.append(info)
        else: # 겹치는 호텔의 경우 -> cost만 업데이트 한다
            overlapped_hotel_cost_data.append((info[3], info[0]))
    print("overlapped_hotel_cost_data", overlapped_hotel_cost_data)
    print("new_hotel_data", new_hotel_data)
    curs.executemany(sql_insert_hotels_information, new_hotel_data)
    curs.executemany(sql_update_price, overlapped_hotel_cost_data)
    
    #curs.execute(sql_delete) # 테이블 삭제
    #data = curs.fetchall() # 가져오기 (select에서 사용)
    #print(data)
    
    conn.commit() # RDS에 반영하기
    
def dataOverlapChecker(base_data, checking_info):
    '''
    겹치는 데이터가 있는지 여부를 확인하는 메서드
    '''
    
    if (checking_info[0],) in base_data:
        return True
    return False
            

'''
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"usa_newyork" : 60763, "cityId" : 2},
{"france_paris" : 187147, "cityId" : 1},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''

location_id_list = [[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
 
    
for offset in range(360, 420, 30): #그냥돌리면 안되게 방지로 주석해둠
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        hotel_info_list = hotelInformationGetter(location_id, offset)  
        sqlController(hotel_info_list)
    
    
# hotel_info_list = hotelInformationGetter(187147)
# sqlController(hotel_info_list)

[298184, 3]
offset:  360
info ['Keio Presso Inn Tokyo Station Yaesu', 'hotel', 3, 130, 3]
info ['Futakotamagawa Excel Hotel Tokyu', 'hotel', 3, 134, 3]
info ['the b tokyo roppongi', 'hotel', 3, 80, 3]
info ['Grand Central Hotel', 'hotel', 3, 61, 3]
info ['Hotel Amanek Ginza East', 'hotel', 3, 64, 3]
info ['Hotel Dormy Inn Akihabara', 'hotel', 3, 82, 3]
info ['APA Hotel Yamanote Otsuka-Eki Tower', 'hotel', 3, 49, 3]
info ['Hotel Tetora Akabane', 'hotel', 2, 27, 3]
info ['Smile Hotel Tokyo Nihombashi', 'hotel', 3, 48, 3]
info ['Hotel Villa Fontaine Grand Tamachi', 'hotel', 3, 66, 3]
info ['Hotel Princess Garden', 'hotel', 4, 92, 3]
info ['Relief Premium Haneda', 'hotel', 3, 56, 3]
info ['Mercure Tokyo Ginza', 'hotel', 4, 147, 3]
info ['Hotel Fukuracia Harumi', 'hotel', 3, 71, 3]
info ['Sotetsu Fresa Inn Shimbashi Hibiyaguchi', 'hotel', 3, 71, 3]
info ['Ebisuholic Hotel', 'hotel', 3, 165, 3]
info ['Flexstay Inn Ekoda', 'hotel', 2, 58, 3]
info ['Claska', 'hotel', 3, 210, 3]
info ['Hotel Ro

info ['Enjoy Hostel', 'hostel', 2, 28, 1]
info ['Hotel Trianon Rive Gauche', 'hotel', 4, 146, 1]
info ['Normandy Hotel', 'hotel', 4, 121, 1]
info ['Hotel du Ministere', 'hotel', 4, 166, 1]
info ['Hotel de France Gare de Lyon Bastille', 'hotel', 3, 87, 1]
info ['Hotel Pavillon Bastille', 'hotel', 3, 104, 1]
info ['Hotel Gerando', 'hotel', 2, 65, 1]
info ["Hotel d'Amiens", 'hotel', 3, 87, 1]
info ['Le Cinq Codet', 'hotel', 5, 292, 1]
info ['Ibis Paris Avenue de la Republique', 'hotel', 3, 96, 1]
info ['Lenox Montparnasse', 'hotel', 3, 125, 1]
info ['Hotel Balzac', 'hotel', 5, 252, 1]
info ['Hotel Francois 1er', 'hotel', 4, 200, 1]
info ['Hotel Grenelle', 'hotel', 3, 82, 1]
info ['Hotel Provinces Opera', 'hotel', 3, 95, 1]
info ['Ibis Paris Gare de Lyon Ledru Rollin 12eme', 'hotel', 3, 98, 1]
info ['Hotel Molitor Paris - MGallery Hotel Collection', 'hotel', 5, 247, 1]
info ['Hotel Choiseul Opera', 'hotel', 3, 103, 1]
info ['Hotel Parisiana', 'hotel', 3, 85, 1]
info ['Hotel des Deux Contin

info ['The Rooster Cafe Dormitory & Studio', 'hostel', 2, 10, 5]
info ['Sekong Hotel Danang', 'hotel', 3, 23, 5]
info ['Louis Hotel', 'small_hotel', 2, 18, 5]
info ['Tay Bac Hotel', 'hotel', 3, 21, 5]
info ['Phi Yen Hotel', 'small_hotel', 2, 11, 5]
info ['Stingray Hostel', 'hostel', 2, 35, 5]
info ['Pandora Boutique Da Nang', 'hotel', 3, 20, 5]
info ['Cani Beach House', 'small_hotel', 3, 40, 5]
info ['Hava Hotel', 'hotel', 2, 11, 5]
info ['La Lune Hotel', 'small_hotel', 2, 11, 5]
info ['Danang Beach Homestay', 'small_hotel', 2, 11, 5]
info ['Kua Casa Apartment', 'hotel', 3, 29, 5]
info ['Kingdom Hotel', 'hotel', 3, 11, 5]
info ['Dai Long Hotel', 'hotel', 2, 18, 5]
info ['Nam Anh Hotel', 'small_hotel', 2, 12, 5]
info ['An Nam Hotel', 'small_hotel', 2, 13, 5]
info ['Shara Hotel Da Nang', 'hotel', 3, 23, 5]
info ['Seven Sea Hotel Da Nang', 'hotel', 3, 56, 5]
info ['Sen Boutique Villa Apartment', 'guest_house', 3, 25, 5]
info ['NANG Capsule hostel', 'hostel', 2, 14, 5]
info ['CN Palace Bou

info ['Resort Cebu', 'small_hotel', 3, 18, 6]
info ['Punta House', 'guest_house', 2, 33, 6]
info ['Casa Del Mar Beach Hotel', 'hotel', 3, 41, 6]
info ['GV Hotel Lapu-Lapu', 'hotel', 1, 14, 6]
info ['Capitol Tourist Inn', 'hostel', 3, 11, 6]
info ['Bamboo House Resort Moalboal', 'hostel', 0, 16, 6]
info ['Days Hotel by Wyndham Cebu-Toledo', 'hotel', 3, 40, 6]
info ["Hisoler's Beach Resort", 'small_hotel', 2, 47, 6]
overlapped_hotel_cost_data []
new_hotel_data [['Resort Cebu', 'small_hotel', 3, 18, 6], ['Punta House', 'guest_house', 2, 33, 6], ['Casa Del Mar Beach Hotel', 'hotel', 3, 41, 6], ['GV Hotel Lapu-Lapu', 'hotel', 1, 14, 6], ['Capitol Tourist Inn', 'hostel', 3, 11, 6], ['Bamboo House Resort Moalboal', 'hostel', 0, 16, 6], ['Days Hotel by Wyndham Cebu-Toledo', 'hotel', 3, 40, 6], ["Hisoler's Beach Resort", 'small_hotel', 2, 47, 6]]


In [33]:
########## 5성급만 뽑도록!! -> 근데 바꾼거로는 안해도 ㄱㅊ을듯?

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser


def moneyTrimmer(input_string):
    
    '''
    Hotel Price data would look like "$263 - $505".
    This method change the one to 263. (get minimum price)
    '''
    
    temp_list = input_string.split(" - ")
    return (temp_list[0])[1:].replace(",", "")


def hotelInformationGetter(input_data, offset):
    
    '''
    Get hotel data and extract information.
    '''
    
    # 파라미터 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")
    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
    api_url = ("/hotels/list?offset="
                 +str(offset)
                 +"&currency=USD&child_rm_ages=7%252C10&limit=30&checkin=2020-01-13&hotel_class=1&order=asc&lang=en_US&sort=recommended&nights=1&location_id="
                 + str(location_id)
                 +"&adults=1&rooms=1")
    
    conn.request("GET", api_url, headers=headers)
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    hotels_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            hotel_data = []
            hotel_data.append(one_data.get("name"))
            sub_category = one_data.get("subcategory_type")
            sub_categorys = ["hotel", "small_hotel", "hostel", "guest_house", "villa"]
            if not sub_category in sub_categorys:
                continue
            else:
                hotel_data.append(sub_category)                
            hotel_data.append(int(float(one_data.get("hotel_class"))))
            hotel_data.append(int(moneyTrimmer(one_data.get("price"))))
            hotel_data.append(cityId) # cityId
            hotels_data.append(hotel_data)
        except AttributeError as e:
            continue

    return hotels_data

def sqlController(hotel_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password,
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Hotel"
    sql_insert_hotel_information = "INSERT INTO City (continent, country, name) VALUES('Europe', 'France', 'Paris')"
    sql_insert_hotels_information = "INSERT INTO Hotel (name, category, subCategory, cost, CityId) VALUES(%s, %s, %s, %s, %s)"
    sql_delete = "DELETE FROM Hotel"
    sql_update_price = "UPDATE Hotel SET cost = %s WHERE name = %s"
    
    # 호텔 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    hotel_data = curs.fetchall() 
    #print("hotel_data", hotel_data) # 지금까지 DB에 있는 호텔 데이터
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 호텔 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_hotel_data = []
    overlapped_hotel_cost_data = []
    
    for info in hotel_info_list:
        if not dataOverlapChecker(hotel_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_hotel_data.append(info)
        else: # 겹치는 호텔의 경우 -> cost만 업데이트 한다
            overlapped_hotel_cost_data.append((info[3], info[0]))
    print("overlapped_hotel_cost_data", overlapped_hotel_cost_data)
    print("new_hotel_data", new_hotel_data)
    curs.executemany(sql_insert_hotels_information, new_hotel_data)
    curs.executemany(sql_update_price, overlapped_hotel_cost_data)
    
    #curs.execute(sql_delete) # 테이블 삭제
    #data = curs.fetchall() # 가져오기 (select에서 사용)
    #print(data)
    
    conn.commit() # RDS에 반영하기
    
def dataOverlapChecker(base_data, checking_info):
    '''
    겹치는 데이터가 있는지 여부를 확인하는 메서드
    '''
    
    if (checking_info[0],) in base_data:
        return True
    return False
            

'''
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"usa_newyork" : 60763, "cityId" : 2},
{"france_paris" : 187147, "cityId" : 1},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''

location_id_list = [[298085, 5]]
 
    
for offset in range(90, 150, 30): #그냥돌리면 안되게 방지로 주석해둠
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        hotel_info_list = hotelInformationGetter(location_id, offset)  
        sqlController(hotel_info_list)


[298085, 5]
offset:  90
overlapped_hotel_cost_data []
new_hotel_data []
[298085, 5]
offset:  120
overlapped_hotel_cost_data []
new_hotel_data []


In [8]:
# 식당 정보 기본으로 넣기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

def moneyTrimmer(input_string):
    '''
    Food Price data would look like "$20 - $40".
    This method change the one to 30.
    This element will be inserted in cost column.
    '''
    temp_list = input_string.replace(",", "").split(" - ")
    if len(temp_list) == 1:
        return int(temp_list[0][1:])
    mean = (int)((int(temp_list[0][1:])+int(temp_list[1][1:]))/2)
    
    return mean

def priceLevelTrimmer(input_price_level):
    '''
    Price level data would look like $$ - $$$.
    This method change the one to 2.
    $ -> 1
    $$ - $$$ -> 2
    $$$$ -> 3
    This element will be inserted in grade column.
    '''
    if input_price_level == None:
        return None
    
    if input_price_level.count("$") == 1:
        return 1
    elif input_price_level.count("$") == 5:
        return 2
    else:
        return 3
    
def restaurantInformationGetter(input_data, offset):
    '''
    Get restaurant data and extract information.
    '''
    
    # 파라미터 변수 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
        
    conn.request("GET", "/restaurants/list?restaurant_tagcategory_standalone=10591&lunit=km&restaurant_tagcategory=10591&limit=30&prices_restaurants=10953%252C10955&restaurant_mealtype=10598%252C10599&currency=USD&lang=en_US&offset="
                 + str(offset)
                 + "&location_id="
                 + str(location_id),
                 headers=headers)

    # prices_restaurants=10953%252C10954&  -> 1과 3단계만 넣기.
    
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    restaurants_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            restaurant_data = []
            if one_data.get("name") == None:
                continue
            restaurant_data.append(one_data.get("name"))
            restaurant_data.append(priceLevelTrimmer(one_data.get("price_level"))) # grade
            if -1 == one_data.get("price", -1): # cost가 없으면 그 데이터는 넣지 않는다.
                continue
            elif 1 == int(moneyTrimmer(one_data.get("price"))): # cost가 1이면 안넣기
                continue
            else:
                restaurant_data.append(int(moneyTrimmer(one_data.get("price")))) # 넣기
            restaurant_data.append(cityId) # cityId
            restaurants_data.append(restaurant_data)
        except AttributeError as e:
            continue
    return restaurants_data

def sqlController(food_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user ,password=db_password, 
                           db=db_name, charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Food"
    sql_insert_foods_information = "INSERT INTO Food (name, grade, cost, CityId) VALUES(%s, %s, %s, %s)"
    sql_delete = "DELETE FROM Food"
    sql_update_price = "UPDATE Food SET cost = %s WHERE name = %s"
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    food_data = curs.fetchall() 
    #print("food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_food_data = []
    overlapped_food_cost_data = []
    for info in food_info_list:
        if not dataOverlapChecker(food_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_food_data.append(info)
        else: # 겹치는 food의 경우 -> cost만 업데이트 한다
            overlapped_food_cost_data.append((info[3], info[0]))
    print("overlapped_food_cost_data", overlapped_food_cost_data)
    print("new_food_data", new_food_data)
    curs.executemany(sql_insert_foods_information, new_food_data)
    curs.executemany(sql_update_price, overlapped_food_cost_data)

    conn.commit() # RDS에 반영하기


'''
{"france_paris" : 187147, "cityId" : 1},
{"usa_newyork" : 60763, "cityId" : 2},
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''
location_id_list = [[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]


for offset in range(420, 480, 30):
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        restaurant_info_list = restaurantInformationGetter(location_id, offset)  
        sqlController(restaurant_info_list)
    

[298184, 3]
offset:  420
info ['Hirata Bokujo Tokyo Midtown', 2, 18, 3]
info ['Rikyu Tokyo Solamachi', 2, 13, 3]
info ['Beer Hall Lion Ginza 7Chome', 2, 18, 3]
info ['Shinahanten', 2, 18, 3]
info ['Singapore Hainanchifan Shiodome City Center', 2, 8, 3]
info ['Sushikuni', 2, 38, 3]
info ['Ginza Sushi Marui Sohonten', 2, 18, 3]
info ['Tendan Ginza', 2, 18, 3]
info ['Coconoma Season Dining', 2, 37, 3]
info ['Sizzler Tokyo Dome Hotel', 2, 18, 3]
info ['Shokusai Kadota', 2, 41, 3]
overlapped_food_cost_data []
new_food_data [['Hirata Bokujo Tokyo Midtown', 2, 18, 3], ['Rikyu Tokyo Solamachi', 2, 13, 3], ['Beer Hall Lion Ginza 7Chome', 2, 18, 3], ['Shinahanten', 2, 18, 3], ['Singapore Hainanchifan Shiodome City Center', 2, 8, 3], ['Sushikuni', 2, 38, 3], ['Ginza Sushi Marui Sohonten', 2, 18, 3], ['Tendan Ginza', 2, 18, 3], ['Coconoma Season Dining', 2, 37, 3], ['Sizzler Tokyo Dome Hotel', 2, 18, 3], ['Shokusai Kadota', 2, 41, 3]]
[293913, 4]
offset:  420
info ['Da Dao Cheng Braised Pork On Ri

info ['Fraunces Tavern', 2, 30, 2]
info ['Wagamama', 2, 20, 2]
info ['SuperNova', 2, 20, 2]
info ['Bread & Tulips', 2, 18, 2]
info ['Hundred Acres', 2, 35, 2]
info ['Joe Allen', 2, 25, 2]
info ['Tout Va Bien', 2, 35, 2]
info ['Merchants River House', 2, 21, 2]
info ['Cielo at the Mayfair', 2, 32, 2]
info ["Lil' Frankie's Pizza", 2, 30, 2]
info ['Obica Mozzarella Bar - Flatiron', 2, 60, 2]
info ['Cafe China', 2, 50, 2]
info ['Gigino Wagner Park', 2, 22, 2]
info ['New Wonjo Restaurant', 2, 45, 2]
info ['Elmo', 2, 20, 2]
info ['The Stagecoach Tavern', 2, 21, 2]
info ['Chai Thai Kitchen', 2, 12, 2]
info ['Cupping Room Cafe', 2, 20, 2]
info ['Inakaya', 2, 65, 2]
info ['Topaz Thai Restaurant', 2, 30, 2]
info ['Hide-Chan Ramen', 1, 20, 2]
info ['Pera Soho', 2, 40, 2]
info ["Bubby's", 2, 25, 2]
overlapped_food_cost_data []
new_food_data [['Fraunces Tavern', 2, 30, 2], ['Wagamama', 2, 20, 2], ['SuperNova', 2, 20, 2], ['Bread & Tulips', 2, 18, 2], ['Hundred Acres', 2, 35, 2], ['Joe Allen', 2, 25

In [10]:
# 식당 3 등급만 넣기.

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

def moneyTrimmer(input_string):
    '''
    Food Price data would look like "$20 - $40".
    This method change the one to 30.
    This element will be inserted in cost column.
    '''
    temp_list = input_string.replace(",", "").split(" - ")
    if len(temp_list) == 1:
        return int(temp_list[0][1:])
    mean = (int)((int(temp_list[0][1:])+int(temp_list[1][1:]))/2)
    
    return mean

def priceLevelTrimmer(input_price_level):
    '''
    Price level data would look like $$ - $$$.
    This method change the one to 2.
    $ -> 1
    $$ - $$$ -> 2
    $$$$ -> 3
    This element will be inserted in grade column.
    '''
    if input_price_level == None:
        return None
    
    if input_price_level.count("$") == 1:
        return 1
    elif input_price_level.count("$") == 5:
        return 2
    else:
        return 3
    
def restaurantInformationGetter(input_data, offset):
    '''
    Get restaurant data and extract information.
    '''
    
    # 파라미터 변수 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
        
    conn.request("GET", "/restaurants/list?lunit=km&limit=30&prices_restaurants=10953&currency=USD&offset="
                 + str(offset)
                 + "&lang=en_US&location_id="
                 + str(location_id),
                 headers=headers)
    
    # prices_restaurants=10953%252C10954&  -> 1과 3단계만 넣기. 지금은 3단계만 넣는 것으로 되어있음. 그리고 저 주석이 맞는지 모르겠음.
    
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    restaurants_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            restaurant_data = []
            if one_data.get("name") == None:
                continue
            restaurant_data.append(one_data.get("name"))
            restaurant_data.append(priceLevelTrimmer(one_data.get("price_level"))) # grade
            
            if -1 == one_data.get("price", -1): # cost가 없으면 그 데이터는 넣지 않는다.
                continue
            elif 1 == int(moneyTrimmer(one_data.get("price"))): # cost가 1이면 안넣기
                continue
            else:
                restaurant_data.append(int(moneyTrimmer(one_data.get("price")))) # 넣기
                
            restaurant_data.append(cityId) # cityId
            restaurants_data.append(restaurant_data)
        except AttributeError as e:
            continue
    return restaurants_data

def sqlController(food_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user ,password=db_password, 
                           db=db_name, charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Food"
    sql_insert_foods_information = "INSERT INTO Food (name, grade, cost, CityId) VALUES(%s, %s, %s, %s)"
    sql_delete = "DELETE FROM Food"
    sql_update_price = "UPDATE Food SET cost = %s WHERE name = %s"
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    food_data = curs.fetchall() 
    #print("food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_food_data = []
    overlapped_food_cost_data = []
    for info in food_info_list:
        if not dataOverlapChecker(food_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_food_data.append(info)
        else: # 겹치는 food의 경우 -> cost만 업데이트 한다
            overlapped_food_cost_data.append((info[3], info[0]))
    print("overlapped_food_cost_data", overlapped_food_cost_data)
    print("new_food_data", new_food_data)
    curs.executemany(sql_insert_foods_information, new_food_data)
    curs.executemany(sql_update_price, overlapped_food_cost_data)

    conn.commit() # RDS에 반영하기


'''
{"france_paris" : 187147, "cityId" : 1},
{"usa_newyork" : 60763, "cityId" : 2},
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''
location_id_list = [[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]

for offset in range(0, 120, 30):
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        restaurant_info_list = restaurantInformationGetter(location_id, offset)  
        sqlController(restaurant_info_list)
    

[298184, 3]
offset:  0
info ['Maruka', 1, 8, 3]
info ['Berg', 1, 8, 3]
info ['Takase Ikebukuro Honten', 1, 8, 3]
info ['Bistro Hitsujiya', 1, 8, 3]
overlapped_food_cost_data [(3, 'Sakura Cafe Jimbocho'), (3, 'Ichiran, Shibuya'), (3, 'IPPUDO Roppongi'), (3, 'Yoshinoya Harajuku Takeshita Gate'), (3, 'Harajuku Gyozaro'), (3, 'ABC Ramen'), (3, 'Nakajima'), (3, 'Al Dente Shinjuku')]
new_food_data [['Maruka', 1, 8, 3], ['Berg', 1, 8, 3], ['Takase Ikebukuro Honten', 1, 8, 3], ['Bistro Hitsujiya', 1, 8, 3]]
[293913, 4]
offset:  0
info ['Osteria By Vincent', 1, 10, 4]
info ['The Local Craft Beer House', 1, 17, 4]
overlapped_food_cost_data [(4, 'Gusto Pizza')]
new_food_data [['Osteria By Vincent', 1, 10, 4], ['The Local Craft Beer House', 1, 17, 4]]
[298085, 5]
offset:  0
overlapped_food_cost_data [(5, 'Horizon Bar'), (5, 'Bikini Bottom'), (5, 'Cabanon Palace'), (5, 'Taco Ngon'), (5, 'Tacos Danang'), (5, 'Monsieur Crepes'), (5, "Veda's Kitchen"), (5, 'Tamarind Tree Da Nang Restaurant'), (5, 'Mah

info ['Non Solo Piada', 1, 9, 2]
info ['Fat Cat', 1, 7, 2]
info ['Wo Hop', 1, 20, 2]
info ['Morning Star Cafe', 1, 15, 2]
info ['Lexington Candy Shop', 1, 15, 2]
info ["Freddie and Pepper's", 1, 15, 2]
info ['Judge Roy Bean Public House', 1, 15, 2]
info ['Cafe Cranberry', 1, 7, 2]
info ["EJ's Luncheonette", 1, 20, 2]
overlapped_food_cost_data []
new_food_data [['Non Solo Piada', 1, 9, 2], ['Fat Cat', 1, 7, 2], ['Wo Hop', 1, 20, 2], ['Morning Star Cafe', 1, 15, 2], ['Lexington Candy Shop', 1, 15, 2], ["Freddie and Pepper's", 1, 15, 2], ['Judge Roy Bean Public House', 1, 15, 2], ['Cafe Cranberry', 1, 7, 2], ["EJ's Luncheonette", 1, 20, 2]]
[187147, 1]
offset:  60
info ['MEKONG', 1, 7, 1]
info ['Chez Mamane', 1, 16, 1]
info ['Pizza Rossi', 1, 14, 1]
info ["L'Express de Lyon", 1, 18, 1]
info ["Pizz-art l'Atelier della pizza a la coupe", 1, 11, 1]
info ['Mersea Faubourg Montmartre', 1, 17, 1]
info ['Papito', 1, 10, 1]
info ['Les Crocs', 1, 15, 1]
info ['La Catrina', 1, 7, 1]
info ['La Crepe

In [36]:
# 호텔 중앙값 넣기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

# outlier 제거하는 메소드
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]


def hotelMedianController():
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password, 
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql_select_hotel = "select * from Hotel"
    sql_select_median = "select * from Median"
    sql_insert_median_data = "INSERT INTO Median (cityId, category, cost, urlHotel) VALUES(%s, %s, %s, %s)"
    sql_update_median_data = "UPDATE Median SET cost = %s, urlHotel = %s WHERE cityId = %s AND category = %s"
    
    # 테이블에 들어갈 url
    url_paris = "https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html"
    url_newyork = "https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html"
    url_tokyo = "https://www.tripadvisor.co.kr/Hotels-g298184-Tokyo_Tokyo_Prefecture_Kanto-Hotels.html"
    url_taipei = "https://www.tripadvisor.co.kr/Hotels-g293913-Taipei.html"
    url_danang = "https://www.tripadvisor.co.kr/Hotels-g298085-Da_Nang.html"
    url_cebu = "https://www.tripadvisor.co.kr/Hotels-g294261-Cebu_Island_Visayas-Hotels.html"
    urls = [url_paris, url_newyork, url_tokyo, url_taipei, url_danang, url_cebu]
    
    # 테이블에 들어갈 음식점 호칭
    hotel_naming = ["저가호텔", "일반호텔", "고급호텔", "최고급호텔"] # 1성과 2성 모두 저가호텔로하기
    
    # Hotel 테이블의 내용을 가져오기 + 출력
    curs.execute(sql_select_hotel) 
    hotel_data = curs.fetchall() 
    #print("@@@hotel_data", hotel_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    cities = [[], [], [], [], [], []]

    for hotel_info in hotel_data:
        if hotel_info[5] == 1:
            cities[0].append(hotel_info)
        elif hotel_info[5] == 2:
            cities[1].append(hotel_info)
        elif hotel_info[5] == 3:
            cities[2].append(hotel_info)
        elif hotel_info[5] == 4:
            cities[3].append(hotel_info)
        elif hotel_info[5] == 5:
            cities[4].append(hotel_info)
        else:
            cities[5].append(hotel_info)
        
    for city in cities:
        temp_list = [[], [], [], [], []]
        for hotel in city:
            if hotel[3] == "0":
                continue
            elif hotel[3] =="1":
                temp_list[0].append(hotel[4])
            elif hotel[3] =="2":
                temp_list[1].append(hotel[4])
            elif hotel[3] =="3":
                temp_list[2].append(hotel[4])
            elif hotel[3] =="4":
                temp_list[3].append(hotel[4])
            else:
                temp_list[4].append(hotel[4])
        city[:] = temp_list[:]
    #print("cities", cities) # 각 도시별, 등급별 가격이 모임
        
    median = []    
    np_cities = np.array(cities)
    for city_idx, np_city in enumerate(np_cities):
        inner_median = []
        for grade_idx, np_grade in enumerate(np_city):
            inner_median.append(int(round(np.mean(reject_outliers(np.array(np_grade))))))
        median.append(inner_median)
    print("median", median)
        
    # Refactor: 1성과 2성의 평균으로 저가호텔가격을 만들기 (1성이 초저가 였는데 2성과 평균을 내고 초저가 가격은 없엔다))
    hotel_median_infos = []
    for city in median:
        hotel_median_list = [int((city[0] + city[1])/2), *city[2:]]
        hotel_median_infos.append(hotel_median_list)   
        
    # Median 테이블의 내용을 가져오기
    curs.execute(sql_select_median) 
    median_data = curs.fetchall() 
    print("median_data", len(median_data))

    final_median_list = []
    print("len(median_data)", len(median_data) )
    if len(median_data) == 0: # 테이블 비었을 경우 -> insert
        for city_idx, one_city in enumerate(hotel_median_infos):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(city_idx+1)
                temp_list.append(hotel_naming[idx])
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                final_median_list.append(temp_list)  
        print("final_median_list", final_median_list)
        curs.executemany(sql_insert_median_data, final_median_list)
    else: # 테이블 이미 차있는 경우 -> update
        print("median_data", median_data)
        for city_idx, one_city in enumerate(hotel_median_infos):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                temp_list.append(city_idx+1)
                temp_list.append(hotel_naming[idx])
                final_median_list.append(temp_list) 
        print("final_median_list", final_median_list)
        curs.execute("SET SQL_SAFE_UPDATES = 0")
        curs.executemany(sql_update_median_data, final_median_list)
        
    conn.commit() # RDS에 반영하기

    
hotelMedianController()
    

median [[48, 68, 99, 156, 343], [54, 101, 133, 185, 437], [39, 53, 85, 185, 449], [22, 30, 65, 131, 233], [13, 14, 22, 55, 146], [16, 21, 38, 70, 192]]
median_data 24
len(median_data) 24
median_data ((1, 1, '저가호텔', 58, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (2, 1, '일반호텔', 99, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (3, 1, '고급호텔', 156, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (4, 1, '최고급호텔', 343, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (5, 2, '저가호텔', 77, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (6, 2, '일반호텔', 133, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (7, 2, '고급호텔', 185, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (8, 2, '최고급호텔', 437, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', 

In [37]:
# 식당 중앙값 넣기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

# outlier 제거하는 메소드
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]


def foodMedianController():
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password, 
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql_select_food = "select * from Food"
    sql_select_median = "select * from Median"
    sql_insert_median_data = "INSERT INTO Median (cityId, category, cost, urlFood) VALUES(%s, %s, %s, %s)"
    sql_update_median_data = "UPDATE Median SET cost = %s, urlFood = %s WHERE cityId = %s AND category = %s"
    
    # 테이블에 들어갈 url
    url_paris = "https://www.tripadvisor.co.kr/Restaurants-g187147-Paris_Ile_de_France.html"
    url_newyork = "https://www.tripadvisor.co.kr/Restaurants-g60763-New_York_City_New_York.html"
    url_tokyo = "https://www.tripadvisor.co.kr/Restaurants-g298184-Tokyo_Tokyo_Prefecture_Kanto-Hotels.html"
    url_taipei = "https://www.tripadvisor.co.kr/Restaurants-g293913-Taipei.html"
    url_danang = "https://www.tripadvisor.co.kr/Restaurants-g298085-Da_Nang.html"
    url_cebu = "https://www.tripadvisor.co.kr/Restaurants-g294261-Cebu_Island_Visayas.html"
    urls = [url_paris, url_newyork, url_tokyo, url_taipei, url_danang, url_cebu]
    
    # 테이블에 들어갈 음식점 호칭
    restaurant_naming = ["간편식", "일반음식점", "고급음식점"]
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql_select_food) 
    food_data = curs.fetchall() 
    #print("@@@food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    cities = [[], [], [], [], [], []]

    for food_info in food_data:
        if food_info[4] == 1:
            cities[0].append(food_info)
        elif food_info[4] == 2:
            cities[1].append(food_info)
        elif food_info[4] == 3:
            cities[2].append(food_info)
        elif food_info[4] == 4:
            cities[3].append(food_info)
        elif food_info[4] == 5:
            if food_info[3] > 10000:
                continue
            cities[4].append(food_info)
        else:
            cities[5].append(food_info)
    
    for city in cities:
        temp_list = [[], [], []]
        for restaurant in city:
            if restaurant[2] == "1":
                temp_list[0].append(restaurant[3])
            elif restaurant[2] =="2":
                temp_list[1].append(restaurant[3])
            else:
                temp_list[2].append(restaurant[3])
        city[:] = temp_list[:]
        
    median = []    
    np_cities = np.array(cities)
    for idx, np_city in enumerate(np_cities):
        inner_median = []
        for np_grade in np_city:
            inner_median.append(int(round(np.mean(reject_outliers(np.array(np_grade))))))
        #inner_median.append(idx + 1)
        median.append(inner_median)
    print(median)
        
    # Median 테이블의 내용을 가져오기
    curs.execute(sql_select_median) 
    median_data = curs.fetchall() 
    print("median_data", len(median_data))

    final_median_list = []
    if len(median_data) == 24: # 테이블 비었을 경우 -> insert
        for city_idx, one_city in enumerate(median):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(city_idx+1)
                temp_list.append(restaurant_naming[idx])
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                final_median_list.append(temp_list)  
        print("final_median_list", final_median_list)
        curs.executemany(sql_insert_median_data, final_median_list)
    else: # 테이블 이미 차있는 경우 -> update
        print("median_data", median_data)
        for city_idx, one_city in enumerate(median):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                temp_list.append(city_idx+1)
                temp_list.append(restaurant_naming[idx])
                final_median_list.append(temp_list) 
        curs.executemany(sql_update_median_data, final_median_list)

    conn.commit() # RDS에 반영하기

    
foodMedianController()
    

[[7, 29, 84], [9, 28, 56], [7, 26, 67], [8, 79, 76], [5, 15, 91], [5, 16, 126]]
median_data 24
final_median_list [[1, '간편식', 7, 'https://www.tripadvisor.co.kr/Restaurants-g187147-Paris_Ile_de_France.html'], [1, '일반음식점', 29, 'https://www.tripadvisor.co.kr/Restaurants-g187147-Paris_Ile_de_France.html'], [1, '고급음식점', 84, 'https://www.tripadvisor.co.kr/Restaurants-g187147-Paris_Ile_de_France.html'], [2, '간편식', 9, 'https://www.tripadvisor.co.kr/Restaurants-g60763-New_York_City_New_York.html'], [2, '일반음식점', 28, 'https://www.tripadvisor.co.kr/Restaurants-g60763-New_York_City_New_York.html'], [2, '고급음식점', 56, 'https://www.tripadvisor.co.kr/Restaurants-g60763-New_York_City_New_York.html'], [3, '간편식', 7, 'https://www.tripadvisor.co.kr/Restaurants-g298184-Tokyo_Tokyo_Prefecture_Kanto-Hotels.html'], [3, '일반음식점', 26, 'https://www.tripadvisor.co.kr/Restaurants-g298184-Tokyo_Tokyo_Prefecture_Kanto-Hotels.html'], [3, '고급음식점', 67, 'https://www.tripadvisor.co.kr/Restaurants-g298184-Tokyo_Tokyo_Prefectur

In [70]:
# 연습

# {"japan_tokyo_shinjuku" : 1066457, "cityId" : 2},
# {"japan_tokyo_sibuya" : 1066456, "cityId" : 2},
# {"taiwan_taipei" : 293913, "cityId" : 3},
# {"vietnam_danang" : 298085, "cityId" : 4},
# {"usa_newyork" : 60763, "cityId" : 1},
# {"france_paris" : 187147, "cityId" : 0},
# {"philippines_cebu" : 298460, "cityId" : 5}

location_id_dict = [[1066457, 2], [1066456, 2], [293913, 3], [298085, 4], [60763, 1], [187147, 0], [298460, 5]]

    
for location_id in location_id_dict:
    print(location_id)
    # hotel_info_list = hotelInformationGetter(location_id)
    # sqlController(hotel_info_list)
    

[1066457, 2]
[1066456, 2]
[293913, 3]
[298085, 4]
[60763, 1]
[187147, 0]
[298460, 5]


In [70]:
# 연결 연습

import requests
import json
import http.client
import numpy as np
import pymysql

'''
Get restaurant data and extract information.
'''
# rapidapi 연결하기
conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
    'x-rapidapi-key': "164b57de3amsh8a6777213843ef5p15508djsn556f2053e4ab"
    }

# 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
offset_number = 0

conn.request("GET", "/restaurants/list?restaurant_tagcategory_standalone=10591&currency=KRW&lunit=km&restaurant_tagcategory=10591&limit=30&prices_restaurants=10953%252C10955&restaurant_mealtype=10598%252C10599&currency=USD&lang=en_US&offset="
             + str(30)
             + "&location_id="
             + str(298085),
             headers=headers)

res = conn.getresponse()

# response 읽기
data = res.read()

# response를 dictionary로 만들기
dict_data = json.loads(data)
print(dict_data) # 확인 출력문

# 딕셔너리 다듬기
dict_data_data = dict_data.get('data')


{'data': [{'location_id': '7057425', 'name': 'Limoncello', 'latitude': '16.066114', 'longitude': '108.22364', 'num_reviews': '895', 'timezone': 'Asia/Ho_Chi_Minh', 'location_string': 'Da Nang', 'photo': {'images': {'small': {'width': '150', 'url': 'https://media-cdn.tripadvisor.com/media/photo-l/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '150'}, 'thumbnail': {'width': '50', 'url': 'https://media-cdn.tripadvisor.com/media/photo-t/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '50'}, 'original': {'width': '1280', 'url': 'https://media-cdn.tripadvisor.com/media/photo-o/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '853'}, 'large': {'width': '1024', 'url': 'https://media-cdn.tripadvisor.com/media/photo-w/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '682'}, 'medium': {'width': '550', 'url': 'https://media-cdn.tripadvisor.com/media/photo-s/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '367'}}, 'is_blessed': True, 'uploaded_date': '2018-02-23T05:17:09-0500', 'caption': 